In [1]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc", cache_dir="K:\hugging-hub-cache")
raw_datasets

<>:3: SyntaxWarning: invalid escape sequence '\h'
<>:3: SyntaxWarning: invalid escape sequence '\h'
C:\Users\jhyan\AppData\Local\Temp\ipykernel_38544\3693815070.py:3: SyntaxWarning: invalid escape sequence '\h'
  raw_datasets = load_dataset("glue", "mrpc", cache_dir="K:\hugging-hub-cache")
C:\Users\jhyan\.conda\envs\llm-learn\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
C:\Users\jhyan\.conda\envs\llm-learn\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 1725/1725 [00:00<00:00, 428624.08 examples/s]


DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [17]:
raw_datasets["train"]["sentence1"]
print("------------")
len(list(raw_datasets["train"]["sentence2"]))

------------


3668

In [27]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokenized_dataset = tokenizer(
    list(raw_datasets["train"]["sentence1"]),
    list(raw_datasets["train"]["sentence2"]),
    padding=True,
    truncation=True,
)
type(tokenized_dataset)

transformers.tokenization_utils_base.BatchEncoding

In [32]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"],
                     truncation=True)

In [33]:
tokenized_dataset = raw_datasets.map(tokenize_function, batched=True)
tokenized_dataset

Map: 100%|██████████| 1725/1725 [00:00<00:00, 22236.69 examples/s]


DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [34]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [39]:
samples = tokenized_dataset["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]


[50, 59, 47, 67, 59, 50, 62, 32]

In [40]:
batch = data_collator(samples)

In [41]:
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 67]),
 'token_type_ids': torch.Size([8, 67]),
 'attention_mask': torch.Size([8, 67]),
 'labels': torch.Size([8])}